In [10]:
from bs4 import BeautifulSoup
import requests, pandas as pd

In [2]:
%pip install bs4

Defaulting to user installation because normal site-packages is not writeable
  Using cached bs4-0.0.2-py2.py3-none-any.whl.metadata (411 bytes)
Using cached bs4-0.0.2-py2.py3-none-any.whl (1.2 kB)
   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -------------------------------------- - 143.4/147.9 kB 4.3 MB/s eta 0:00:01
   ---------------------------------------- 147.9/147.9 kB 2.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/64.9 kB ? eta -:--:--
   ---------------------------------------- 64.9/64.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/167.3 kB ? eta -:--:--
   ---------------------------------------- 167.3/167.3 kB 5.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/100.4 kB ? eta -:--:--
   ---------------------------------------- 100.4/100.4 kB 6.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/66.9 kB ? eta -:--:--
   ---------------------------------------- 66.9/66.9 kB ? eta 0:00:00
   ---------------------------------------- 0.0/121.4 kB ? eta -:--:--
   ---------------------------------------- 121.4/121.4 kB 7.0 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
link="https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)"

**Available Parsers in BeautifulSoup:**

    html.parser:

        Built into Python's standard library.
        Fast, but may not be as lenient with malformed HTML.
    lxml:

        Requires the lxml library.
        Very fast, robust, and supports both HTML and XML.
        More tolerant of poorly formed HTML.
    html5lib:

        Requires the html5lib library.
        Parses HTML exactly like a web browser would, following the HTML5 specification.
        Slowest, but the most lenient and handles malformed HTML very well.
    xml:

        Built into Python, but more suited for XML documents.
        Strict with XML content, may not handle HTML well.

In [11]:
# using html.parser to extract all the text as a parser tree

try:
    htmlpage_response=requests.get(link)
    # print(htmlpage_response.text)
    soup=BeautifulSoup(htmlpage_response.text,'html.parser')
    # print(soup)
except Exception as e:
    print(e)



In [12]:
# Find the first element with the class 'intro'
Reference_content = soup.find(class_='mw-references-wrap')

print(Reference_content.contents)  # Output: This is an introductory paragraph.



[<ol class="references">
<li id="cite_note-China-THM-1"><span class="mw-cite-backlink">^ <a href="#cite_ref-China-THM_1-0"><sup><i><b>a</b></i></sup></a> <a href="#cite_ref-China-THM_1-1"><sup><i><b>b</b></i></sup></a> <a href="#cite_ref-China-THM_1-2"><sup><i><b>c</b></i></sup></a></span> <span class="reference-text">Figures for China exclude <a href="/wiki/Taiwan" title="Taiwan">Taiwan</a>, and the <a href="/wiki/Special_administrative_regions_of_China" title="Special administrative regions of China">special administrative regions</a> of <a href="/wiki/Hong_Kong" title="Hong Kong">Hong Kong</a> and <a href="/wiki/Macau" title="Macau">Macau</a>.</span>
</li>
<li id="cite_note-3"><span class="mw-cite-backlink"><b><a href="#cite_ref-3">^</a></b></span> <span class="reference-text">Based on the best available GDP figure for each country at the time of creation (16 June 2024). Best available GDP figure was defined as the latest available figure from the <a href="/wiki/International_Moneta

In [13]:
ref_list_content = []
listcount=1
for li in Reference_content.find_all('li'):
    # ref_list_content.append(li.text.strip())

    # list_of_ref=str(listcount) , li.text.strip().replace("^","")
    reference_text=li.text.strip().replace("^","")
    # print(reference_text)

    ref_list_content.append({
        'Serial_Number': listcount,
        'Reference_Text': reference_text
    })
    
    # list_of_ref=(list(list_of_ref[listcount]).replace(".",""))
    listcount+=1
print(ref_list_content)

[{'Serial_Number': 1, 'Reference_Text': ' a b c Figures for China exclude Taiwan, and the special administrative regions of Hong Kong and Macau.'}, {'Serial_Number': 2, 'Reference_Text': " Based on the best available GDP figure for each country at the time of creation (16 June 2024). Best available GDP figure was defined as the latest available figure from the International Monetary Fund (including forecasts for the current year), otherwise the latest available figure from the World Bank, otherwise the latest available figure from the United Nations Statistics Division; however, countries that didn't have any figure for a year more recent than 2019 were not included at all. For almost all countries, the latest IMF forecast figure for 2024 was used."}, {'Serial_Number': 3, 'Reference_Text': ' Figures exclude the special administrative regions of Hong Kong and Macau.'}, {'Serial_Number': 4, 'Reference_Text': ' The name used in the IMF report is "Taiwan Province of China".'}, {'Serial_Num

In [45]:
df=pd.DataFrame(ref_list_content)
df.head()

,Serial_Number,Reference_Text
0,1,"a b c Figures for China exclude Taiwan, and t..."
1,2,Based on the best available GDP figure for ea...
2,3,Figures exclude the special administrative re...
3,4,"The name used in the IMF report is ""Taiwan Pr..."
4,5,"The name used in the IMF report is ""Hong Kong..."


In [15]:
import mysql.connector

def myDB():
    """Establish a database connection."""
    dbcon = mysql.connector.connect(
        host="192.168.29.244",
        user="test01",
        password="test@123",
        database="a"
    )
    return dbcon

# Create the table
def create_table(cursor):
    create_table_query = """
    CREATE TABLE IF NOT EXISTS references (
        SerialNumber VARCHAR(10) PRIMARY KEY,
        ReferenceText TEXT
    )
    """
    cursor.execute(create_table_query)

# Insert data into the table
def insert_data(cursor, df):
    insert_query = """
    INSERT INTO references (SerialNumber, ReferenceText)
    VALUES (%s, %s)
    """
    for _, row in df.iterrows():
        cursor.execute(insert_query, (row['SerialNumber'], row['ReferenceText']))

# Main process
def main():
    dbcon = myDB()
    cursor = dbcon.cursor()
    
    # Create table
    create_table(cursor)
    
    # Insert data
    insert_data(cursor, df)
    
    # Commit and close connection
    dbcon.commit()
    cursor.close()
    dbcon.close()

if __name__ == "__main__":
    main()



ModuleNotFoundError: No module named '_version'

In [16]:
%pip uninstall mysql-connector-python